In [1]:
import numpy as np
from random import *
import tkinter as tk

%matplotlib inline 
from matplotlib import pyplot as plt

In [6]:
class Game_2048: 
    """
    The board game 2048 Supreme.
    
    Attributes:
    -----------
    board: the game board
    m: height of the board
    n: width of the board
    goal: the target goal to win the game
    prime: the base number of the game
    game_over: 1 = game over, 0 = game ongoing
    win: whether or not we won
    actions: allowable actions at each timestep
    """
    board = np.zeros(4)
    m = 4
    n = 4
    goal = 2048
    prime = 2
    game_over = 0
    win = 0
    actions = ["left", "up", "right", "down", '0', '1', '2', '3', 0, 1, 2, 3]
    
    def __init__(self, m = 4, n = 4, prime = 2, goal = 2048, board_array = None):
        """
        Creates a new instance of the game. Sets up the board, prime number, and goal number. 
        
        By default, it creates a standard 4 x 4 board. 
        
        Inputs
        ------
        m = # rows or the height of the board
        n = # columns or the width of the board
        prime = the base prime number of this game
        goal = the goal number to achieve to win
        board_array = a custom np array to use as a board
        """
        if board_array is None:
            self.board = np.zeros((m, n))
            self.m = m
            self.n = n
        else:
            self.board = board_array
            
        self.goal = goal
        self.prime = prime
        self.game_over = 0
        self.win = 0
        
        for i in range(0, self.prime):
            self.board = self.fill_random_cell(self.board, self.prime)
        
    def fill_random_cell(self, board, prime):
        """
        Fills a random empty cell in the board with the prime value.
        
        Inputs
        ------
        board: board you want to fill a cell with
        prime: the prime number to fill in the board
        
        Outputs:
        --------
        modBoard: the modified board with a cell filled in
        """
        modBoard = np.copy(board)
        i, j = (modBoard == 0).nonzero()
        if i.size != 0:
            rnd = randint(0, i.size - 1)
            modBoard[i[rnd], j[rnd]] = prime
        else:
            self.game_over = 1
        return modBoard
            
    def getSuccessor(self, board):
        """
        Moves all blocks to the left in the given board 
        
        Proceeds by column, moving each item in the column as far left as possible individually.
        
        If a block encounters another block, then it stops unless they are the same block. If so,
        then it counts if we can add a number of those blocks together equal to prime. If so, then
        they are combined. 
        
        Then it fills a random cell on the board. Then checks if it is game over. If a cell could not
        be filled, then it is game over. 
        
        Inputs:
        -------
        board: the board you want to enact moveLeft on
        
        Outputs:
        --------
        modBoard: the left shifted board
        
        """
        n = self.n
        m = self.m
        modBoard = np.copy(board)
        merged = []
        for i in range(n):
            for j in range(m):
                current = modBoard[j][i]
                if current == 0:
                    continue
                y, x = j, i
                for k in range(1, i+1):
                    block = modBoard[j][i-k]
                    if block == 0:
                        modBoard[j][i-k] = current
                        modBoard[y][x] = 0
                        y = j
                        x = i - k
                        continue
                    elif (block != current) or (j, i-k) in merged:
                        break
                    else:
                        count = 1
                        combine = False
                        zeros = [(y, x)]
                        for c in range(k, i+1): 
                            if modBoard[j][i-c] == current:
                                count += 1
                                if count == self.prime: 
                                    modBoard[j][i-c] = current * self.prime
                                    merged.append((j, i-c))
                                    for index in zeros:
                                        modBoard[index[0]][index[1]] = 0
                                    break
                                zeros.append((j, i-c))
                            else:
                                break
        return modBoard
    
    def move(self, direction = 0):
        """
        Moves the board in the desired direction. Moving up, right, or down is just
        moving left on a rotated board. 
        
        Inputs:
        -------
        direction: The direction you wish to move, the following are all equivalent.
            "left" = '0' = 0
            "up" = '1' = 1
            "right" = '2' = 2
            "down" = '3' = 3
        """
        if direction in self.actions:
            board = np.copy(self.board)
            movement = self.actions.index(direction) % 4
            
            board = np.rot90(board, movement)
            board = self.getSuccessor(board)
            board = np.rot90(board, -movement)
            
            board = self.fill_random_cell(board, self.prime)
            self.board = board
        else:
            print("Error. Please enter one of the following: left, up, right, down. \n \
            Alternatively, type: 0, 1, 2, or 3")
    
    def is_game_over(self):
        """
        Checks if the game is over.
        
        Outputs:
        --------
        1: game is over
        0 : game is ongoing
        """
        if self.game_over == 1 or self.is_win() == 1:
            return 1
        else:
            return 0
        
    def is_win(self):
        """
        Checks if we won the game
        
        Outputs:
        --------
        1: we won
        0: we lost
        """
        for block in np.nditer(self.board):
            if block == self.goal:
                return 1
        return 0


In [7]:
game = Game_2048(m = 4, n = 4, prime = 2, goal = 32)
x = np.arange(4).reshape((2,2))

In [8]:
print("Starting game now.")
while True:
    #plt.matshow(game.board)
    #plt.show()
    print(game.board)
    if game.is_game_over() == 1:
        if game.is_win() == 0:
            print("Sorry, you lost.")
        else:
            print("You won!")
        break
    action = input()
    if action.lower() == "exit":
        print("Ending game now.")
        break
    game.move(action)

Starting game now.
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [2. 0. 0. 0.]
 [0. 2. 0. 0.]]
left
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [2. 0. 0. 0.]
 [2. 0. 2. 0.]]
up
[[4. 0. 2. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 2.]
 [0. 0. 0. 0.]]
left
[[4. 2. 2. 0.]
 [0. 0. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 0.]]
left
[[4. 4. 0. 0.]
 [0. 0. 0. 0.]
 [2. 0. 0. 2.]
 [0. 0. 0. 0.]]
left
[[8. 0. 0. 0.]
 [0. 0. 0. 0.]
 [4. 0. 0. 0.]
 [2. 0. 0. 0.]]
right
[[0. 0. 0. 8.]
 [0. 2. 0. 0.]
 [0. 0. 0. 4.]
 [0. 0. 0. 2.]]
left
[[8. 0. 2. 0.]
 [2. 0. 0. 0.]
 [4. 0. 0. 0.]
 [2. 0. 0. 0.]]
right
[[0. 0. 8. 2.]
 [0. 0. 0. 2.]
 [0. 0. 2. 4.]
 [0. 0. 0. 2.]]
up
[[0. 0. 8. 4.]
 [2. 0. 2. 4.]
 [0. 0. 0. 2.]
 [0. 0. 0. 0.]]
up
[[2. 0. 8. 8.]
 [0. 2. 2. 2.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
right
[[ 2.  0.  2. 16.]
 [ 0.  0.  2.  4.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
up
[[ 2.  0.  4. 16.]
 [ 0.  0.  0.  4.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  2.  0.]]
left
[[ 2.  4. 16.  2.]
 [ 4.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 2.  0.  0.  0.]]
right
[[ 2.  4. 16.

In [9]:
""" GUI Interface for the game using Tkinter """
root = tk.Tk()
label = tk.Label(root, text="2048 Supreme")
label.pack()
root.mainloop()